# 4. 메타클래스와 속성

* 메타클래스를 이용하면 파이썬의 class 문을 가로채 클래스가 정의될 때마다 특별한 동작을 제공 가능.
* 강력한 기능이므로 주의해야 함.
    - 동적 속성은 객체들을 오버라이드하다 예상치 못한 부작용을 일으키게 할 수 있음.
    - 메타클래스는 쉽게 이해하기 어려운 극도로 이상한 동작을 만들어내기도 함.
    - 최소 놀람 규칙(rule of least surprise)을 따르고 제대로 이해하고 있는 이디엄 구현시에만 이용할 것.

## Better Way 29. 게터와 세터 메서드 대신에 일반 속성을 사용하자

* **[비추천] 클래스에 게터(getter) 및 세터(setter) 메서드를 명시적으로 구현하는 경우.**

In [1]:
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms

* 사용법은 간단하지만 파이썬답지 않음.

In [2]:
r0 = OldResistor(50e3)
print('Before: %5r' % r0.get_ohms())
r0.set_ohms(10e3)
print('After: %5r' % r0.get_ohms())

Before: 50000.0
After: 10000.0


* 즉석에서 증가시키기 같은 연산에는 사용하기 불편.

In [4]:
r0.set_ohms(r0.get_ohms() + 5e3)

* **파이썬에서는 명시적으로 게터와 세터를 구현할 일이 거의 없음.**
* **항상 간단한 공개 속성부터 구현할 것.**

In [5]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

In [6]:
r1 = Resistor(50e3)
r1.ohms = 10e3

* 즉석에서 증가시키기 같은 연산이 자연스럽고 명확해짐

In [7]:
r1.ohms += 5e3

* **나중에 속성을 설정할 때 특별한 동작이 필요한 경우 @property 데코레이터(decorator)와 이에 대응하는 setter 속성을 사용하는 방법으로 변경 가능.**
    - [예시] Resistor의 새 서브클래스를 정의해 voltage 프로퍼티(property)를 할당하면 current 값이 변경되게 함.
        + 제대로 동작하려면 세터 및 게터 메서드의 이름이 의도한 프로퍼티 이름과 일치해야 함.

In [8]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

* voltage 프로퍼티에 할당하면 voltage 세터 메서드가 실행되어 voltage에 맞게 객체의 current 프로퍼티를 업데이트함.

In [9]:
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After: %5r amps' % r2.current)

Before:     0 amps
After:  0.01 amps


* **프로퍼티에 setter를 설정하면 클래스에 전달된 값들의 타입을 체크하고 값을 검증할 수도 있음.**
    - [예시] 모든 저항값이 0옴보다 큼을 보장하는 클래스 정의.

In [13]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms

+ 속성에 올바르지 않은 저항값을 할당하면 예외 발생.

In [14]:
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: 0.000000 ohms must be > 0

* 생성자에 올바르지 않은 값을 넘겨도 예외 발생.
    - [예시] BoundedResistance.__init__가 self.ohms = -5를 할당하는 Resistor.__init__를 호출하기 때문에 예외 발생. 이 할당문으로 BoundedResistance의 @ohms.setter 메서드가 호출되어 객체 생성이 완료되기도 전에 곧장 검증 코드가 실행됨.

In [15]:
BoundedResistance(-5)

ValueError: -5.000000 ohms must be > 0

* **부모 클래스의 속성을 불변(immutable)으로 만드는 데도 @property를 사용 가능.**

In [16]:
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

* [예시] 객체를 생성하고 나서 프로퍼티에 할당하려고 하면 예외 발생.

In [17]:
r4 = FixedResistance(1e3)
r4.ohms = 2e3

AttributeError: Can't set attribute

* **@property의 가장 큰 단점: 속성에 대응하는 메서드를 서브클래스에서만 공유 가능.**
    - 서로 관련이 없는 클래스는 같은 구현을 공유하지 못함.
    - **파이썬은 디스크립터(descriptor) 또한 지원함.**
        + 디스크립터는 재사용 가능한 프로퍼티 로직 및 다른 많은 쓰임새가 있음(**BW31 참조**).

* **@property 메서드로 세터 및 게터를 구현할 때 예상과 다르게 동작하지 않게 해야 함.**
    - 예를 들면 게터 프로퍼티 메서드에서 다른 속성을 설정하지 말 것.
    - [예시] 게터 프로퍼티 메서드에서 다른 속성을 설정하는 경우.

In [25]:
class MysteriousResistor(Resistor):  
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

In [26]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print('Before: %5r' % r7.voltage)
r7.ohms
print('After: %5r' % r7.voltage)

Before:     0
After:   0.1


* **@property.setter 메서드에서만 관련 객체의 상태를 수정하는 것이 최선의 정책.**
    - **호출하는 쪽이 객체에서 일어날 것이라고 예측하지 못할 만한 다른 부작용은 모두 피해야 함.**
        + [예] 모듈을 동적으로 임포트하기.
        + [예] 느린 헬퍼 함수 실행하기.
        + [예] 비용이 많이 드는 데이터베이스 수행하기.
    - 사용자는 다른 파이썬 객체가 그렇듯 클래스의 속성이 빠르고 쉬우리라 기대할 것.
    - **더 복잡하거나 느린 작업은 일반 메서드로 할 것.**